In [1]:
import carla
import math
import random
import time
import queue
import numpy as np
import cv2
import subprocess
import os
from image_label_generator import ImageLabelGenerator

In [2]:
subprocess.run('taskkill /f /fi "IMAGENAME eq CarlaUE4*"', shell=True)
os.startfile("""C:\carla\CarlaUE4_Low""")
start = input("Pressione enter para iniciar")


In [3]:

client = carla.Client('localhost', 2000)
client.set_timeout(50.0) # seconds
world  = client.load_world('Town01')
bp_lib = world.get_blueprint_library()

# get possible spawn points
current_map = world.get_map()

# spawn_transforms will be a list of carla.Transform
spawn_points = current_map.get_spawn_points()

# spawn vehicle
vehicle_bp =bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])


# # Create a queue to store and retrieve the sensor data
# image_queue = queue.Queue()
# camera.listen(image_queue.put)

In [4]:
# # spawn camera
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_init_trans = carla.Transform(carla.Location(z=2))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)
vehicle.set_autopilot(True)

In [5]:
# # Set up the simulator in synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True # Enables synchronous mode
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

273

In [6]:
# # Create a queue to store and retrieve the sensor data
image_queue = queue.Queue()
camera.listen(image_queue.put)

#### FUNCTINOS THAT CONVERT 3D POINTS INTO CAMERA POINTS

In [7]:
def build_projection_matrix(w, h, fov):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

def get_image_point(loc, K, w2c):
        # Calculate 2D projection of 3D coordinate

        # Format the input coordinate (loc is a carla.Position object)
        point = np.array([loc.x, loc.y, loc.z, 1])
        # transform to camera coordinates
        point_camera = np.dot(w2c, point)

        # New we must change from UE4's coordinate system to an "standard"
        # (x, y ,z) -> (y, -z, x)
        # and we remove the fourth componebonent also
        point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

        # now project 3D->2D using the camera matrix
        point_img = np.dot(K, point_camera)
        # normalize
        point_img[0] /= point_img[2]
        point_img[1] /= point_img[2]

        return point_img[0:2]

#### CALCULATING THE CAMERA PROJECTION

In [8]:

# Get the world to camera matrix
world_2_camera = np.array(camera.get_transform().get_inverse_matrix())

# Get the attributes from the camera
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
fov = camera_bp.get_attribute("fov").as_float()

# Calculate the camera projection matrix to project from 3D -> 2D
K = build_projection_matrix(image_w, image_h, fov)

#### GETTING THE BOUNDING BOXES


In [ ]:
image_label_gen = ImageLabelGenerator(camera, world, 25)
world.tick()
image_label_gen.get_image_and_create_copy_zeros_matrix()

: 

In [11]:
image_label_gen = ImageLabelGenerator(camera, world, 25)
while True:
    
    # Retrieve and reshape the image
    world.tick()
    new_matrix, exists_object, img = image_label_gen.create_label_matrix()
    if exists_object:
        print(new_matrix)
        cv2.imshow('ImageWindowName',img)

: 

In [10]:
bounding_box_set = world.get_level_bbs(carla.CityObjectLabel.TrafficLight)
bounding_box_set.extend(world.get_level_bbs(carla.CityObjectLabel.TrafficSigns))

objects = world.get_environment_objects(carla.CityObjectLabel.TrafficLight)
objects.extend(world.get_environment_objects(carla.CityObjectLabel.TrafficSigns))
edges = [[0,1], [1,3], [3,2], [2,0], [0,4], [4,5], [5,1], [5,7], [7,6], [6,4], [6,2], [7,3]]
image_label_gen = ImageLabelGenerator(camera, world, 25)
while True:
    
    # Retrieve and reshape the image
    world.tick()
    image = image_queue.get()

    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

    # Get the camera matrix 
    world_2_camera = np.array(camera.get_transform().get_inverse_matrix())

    for _object in objects:
        
        bb = _object.bounding_box
        dist = _object.transform.location.distance(camera.get_transform().location)

        # Filter for the vehicles within 50m
        if dist < 25:

        # Calculate the dot product between the forward vector
        # of the vehicle and the vector between the vehicle
        # and the other vehicle. We threshold this dot product
        # to limit to drawing bounding boxes IN FRONT OF THE CAMERA
            forward_vec = camera.get_transform().get_forward_vector()
            ray = _object.transform.location - camera.get_transform().location

            if forward_vec.dot(ray) > 1:
                p1 = get_image_point(bb.location, K, world_2_camera)
                verts = [v for v in bb.get_world_vertices(carla.Transform())]
                x_max = -10000
                x_min = 10000
                y_max = -10000
                y_min = 10000

                for vert in verts:
                    p = get_image_point(vert, K, world_2_camera)
                    # Find the rightmost vertex
                    if p[0] > x_max:
                        x_max = p[0]
                    # Find the leftmost vertex
                    if p[0] < x_min:
                        x_min = p[0]
                    # Find the highest vertex
                    if p[1] > y_max:
                        y_max = p[1]
                    # Find the lowest  vertex
                    if p[1] < y_min:
                        y_min = p[1]

                cv2.line(img, (int(x_min),int(y_min)), (int(x_max),int(y_min)), (0,0,255, 255), 1)
                cv2.line(img, (int(x_min),int(y_max)), (int(x_max),int(y_max)), (0,0,255, 255), 1)
                cv2.line(img, (int(x_min),int(y_min)), (int(x_min),int(y_max)), (0,0,255, 255), 1)
                cv2.line(img, (int(x_max),int(y_min)), (int(x_max),int(y_max)), (0,0,255, 255), 1)
                try:
                    np.savetxt('imagem.csv')
                except:
                    pass


    cv2.imshow('ImageWindowName',img)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [ ]:
# Retrieve and reshape the image
bounding_box_set = world.get_level_bbs(carla.CityObjectLabel.TrafficLight)
bounding_box_set.extend(world.get_level_bbs(carla.CityObjectLabel.TrafficSigns))

objects = world.get_environment_objects(carla.CityObjectLabel.TrafficLight)
objects.extend(world.get_environment_objects(carla.CityObjectLabel.TrafficSigns))
edges = [[0,1], [1,3], [3,2], [2,0], [0,4], [4,5], [5,1], [5,7], [7,6], [6,4], [6,2], [7,3]]

world.tick()
image = image_queue.get()

img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# Get the camera matrix 
world_2_camera = np.array(camera.get_transform().get_inverse_matrix())

for _object in objects:
    
    bb = _object.bounding_box
    dist = _object.transform.location.distance(vehicle.get_transform().location)

    # Filter for the vehicles within 50m
    if dist < 25:

    # Calculate the dot product between the forward vector
    # of the vehicle and the vector between the vehicle
    # and the other vehicle. We threshold this dot product
    # to limit to drawing bounding boxes IN FRONT OF THE CAMERA
        forward_vec = vehicle.get_transform().get_forward_vector()
        ray = _object.transform.location - vehicle.get_transform().location

        if forward_vec.dot(ray) > 1:
            p1 = get_image_point(bb.location, K, world_2_camera)
            verts = [v for v in bb.get_world_vertices(_object.transform)]
            x_max = -10000
            x_min = 10000
            y_max = -10000
            y_min = 10000

            for vert in verts:
                p = get_image_point(vert, K, world_2_camera)
                # Find the rightmost vertex
                if p[0] > x_max:
                    x_max = p[0]
                # Find the leftmost vertex
                if p[0] < x_min:
                    x_min = p[0]
                # Find the highest vertex
                if p[1] > y_max:
                    y_max = p[1]
                # Find the lowest  vertex
                if p[1] < y_min:
                    y_min = p[1]

            cv2.line(img, (int(x_min),int(y_min)), (int(x_max),int(y_min)), (0,0,255, 255), 1)
            cv2.line(img, (int(x_min),int(y_max)), (int(x_max),int(y_max)), (0,0,255, 255), 1)
            cv2.line(img, (int(x_min),int(y_min)), (int(x_min),int(y_max)), (0,0,255, 255), 1)
            cv2.line(img, (int(x_max),int(y_min)), (int(x_max),int(y_max)), (0,0,255, 255), 1)
            try:
                np.savetxt('imagem.csv')
            except:
                pass

cv2.imshow('ImageWindowName',img)


: 